In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
# Libraries
import pandas as pd
import os

from src.data import prep_data as prep
from src.models import model_schemata as schema
from src.models import build_model as build

In [5]:
# os.chdir('/Users/magdalena/OneDrive - Queen Mary, University of London/bezzlab/research/projects/phospho_pi/')
os.chdir('/home/mhuebner/Desktop/bezzlab/research/projects/phospho_pi/')

In [6]:
# reading csv from files into dict
data = {}
data['e_ksea'] = pd.read_csv('data/processed/ebdt_data/sub_network_p/e_ksea.csv')
data['p_fc'] = pd.read_csv('data/processed/ebdt_data/sub_network_p/p_fc_scaled.csv')

In [7]:
e_ksea_ad = prep.expand_ad_cpd(data['e_ksea'], 'value', base=True)
p_fc_ad = prep.expand_ad_cpd(data['p_fc'], 'value', base=True)
e_ksea_ad

,enzyme,sample,tc,value,prob,p_dec,p_inc,p_base
0,ABL1,AC220,10.0,base,0.649536,0.175232,0.175232,0.649536
1,ABL1,AC220,10.0,dec,0.649536,0.649536,0.001000,0.349464
2,ABL1,AC220,10.0,inc,0.649536,0.001000,0.649536,0.349464
3,ABL1,AT13148,10.0,base,0.591795,0.204103,0.204103,0.591795
4,ABL1,AT13148,10.0,dec,0.591795,0.591795,0.001000,0.407205
...,...,...,...,...,...,...,...,...
1459,SRC,Ulixertinib,12.0,dec,0.602812,0.602812,0.001000,0.396188
1460,SRC,Ulixertinib,12.0,inc,0.602812,0.001000,0.602812,0.396188
1461,SRC,Vemurafenib,12.0,base,0.797541,0.101230,0.101230,0.797541
1462,SRC,Vemurafenib,12.0,dec,0.797541,0.797541,0.001000,0.201459


In [8]:
# Mapping data to Problog predicates
# make sure to delete 'base' from schemata and AnnotatedDisjunctionClauseTemplate
predicates = {}
predicates['e_ksea'] = schema.EKseaPredicate()
predicates['e_ksea'].add_data(e_ksea_ad, 'enzyme', 'sample', 'value')
predicates['e_ksea'].add_ad_probabilities(e_ksea_ad, p_dec_col='p_dec', p_base_col='p_base', p_inc_col='p_inc')
predicates['p_fc'] = schema.PFoldChangePredicate()
predicates['p_fc'].add_data(p_fc_ad, 'phosphosite', 'sample', 'value')
predicates['p_fc'].add_ad_probabilities(p_fc_ad, p_dec_col='p_dec', p_base_col='p_base', p_inc_col='p_inc')

In [9]:
model_lfi = 'models/ebdt_data/sub_network/p_model/p_model_lfi_exp12.pl'
for predicate in predicates: 
    ad_generator = build.ProblogStatementGenerator(predicates[predicate])
    clauses = ad_generator.generate_ad_clauses()
    build.insert_statements(model=model_lfi, statements=clauses, location='%% p1::{}'.format(predicate)) # insert into Problog file